In [2]:
import os, sys
import numpy as np
import psycopg2

#### Obtain paths for each colored mask image directory


In [ ]:
masks_dir_path = "../app/assets/images/2-Masks"
mask_dirs = os.listdir(masks_dir_path)
mask_dirs = [os.path.join(masks_dir_path, el) for el in mask_dirs]
fil = [os.path.isdir(el) for el in mask_dirs]

from itertools import compress
mask_dirs = list(compress(mask_dirs, fil))
print mask_dirs

In [55]:
# obtain mask names for every mask image

masks = os.listdir(mask_dirs[0])
masks.sort()
mask_names = [el.split("_")[0] for el in masks]
mask_names = list(set(mask_names))
mask_names = [mask[:-2] if mask.endswith(" -") else mask for mask in mask_names]
mask_names.sort()
print mask_names[:5]


['Diencephalon - Anterior group of the posterior tubercular vmat2 neurons_slice001.png', 'Diencephalon - Anterior group of the posterior tubercular vmat2 neurons_slice002.png', 'Diencephalon - Anterior group of the posterior tubercular vmat2 neurons_slice003.png', 'Diencephalon - Anterior group of the posterior tubercular vmat2 neurons_slice004.png', 'Diencephalon - Anterior group of the posterior tubercular vmat2 neurons_slice005.png', 'Diencephalon - Anterior group of the posterior tubercular vmat2 neurons_slice006.png', 'Diencephalon - Anterior group of the posterior tubercular vmat2 neurons_slice007.png', 'Diencephalon - Anterior group of the posterior tubercular vmat2 neurons_slice008.png', 'Diencephalon - Anterior group of the posterior tubercular vmat2 neurons_slice009.png', 'Diencephalon - Anterior group of the posterior tubercular vmat2 neurons_slice010.png', 'Diencephalon - Anterior group of the posterior tubercular vmat2 neurons_slice011.png', 'Diencephalon - Anterior group 

In [45]:
# create numeric id_dict for mask names
mask_dict = dict()
for i, mask in enumerate(mask_names, 1001):
    mask_dict[mask] = i
mask_dict.keys()[:5]

['Telencephalon - S1181t Cluster',
 'Rhombencephalon - Inferior Olive',
 'Spinal Cord',
 'Rhombencephalon - Caudal Ventral Cluster Labelled by Spinal Backfills',
 'Diencephalon']

In [46]:
# check file size of a single mask image

example_mask_image = os.listdir(mask_dirs[0])[0]
example_mask_image = os.path.join(mask_dirs[0], example_mask_image)
print os.stat(example_mask_image).st_size

391


In [47]:
# prepare postgreSQL table

create_table_command = (
"""
    DROP TABLE masks;
    CREATE TABLE masks (
        mask_img_id SERIAL PRIMARY KEY,
        mask_name TEXT NOT NULL,
        mask_id INT NOT NULL,
        mask_image_path TEXT NOT NULL,
        mask_image_size INT
    )
""")

In [3]:
# connect to zbrain2db database

conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()

In [49]:
# create table masks
try:
    cursor.execute(create_table_command)
    cursor.close()
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print error
finally:
    if conn is not None:
        conn.close()

In [50]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()

In [51]:
# for each mask image create an entry in postgreSQL database

for i, img in enumerate(masks): # masks = os.listdir(mask_dirs[0])
    mask_image_path = img
    mask_name = img.split("_")[0]
    mask_id = mask_dict[mask_name]
    mask_image_size = os.stat(os.path.join(mask_dirs[0], mask_image_path)).st_size
    
    insert_command = ("INSERT INTO masks (mask_img_id, mask_name, mask_id, mask_image_path, mask_image_size) values (%i, '%s', %i, '%s', %i)"
                    % (i, mask_name.replace("'", "''"), mask_id, mask_image_path.replace("'", "''"), mask_image_size))
    
    try:
        cursor.execute(insert_command)
    except Exception, e:
        print repr(e)

In [52]:
cursor.close()
conn.commit()

In [53]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()
cursor.execute('SELECT * FROM masks ORDER BY mask_img_id')

In [54]:
# list entries

rows = cursor.fetchall()
for row in rows[0:10]:
    print row

(0, 'Diencephalon - Anterior group of the posterior tubercular vmat2 neurons', 1002, 'Diencephalon - Anterior group of the posterior tubercular vmat2 neurons_slice001.png', 391)
(1, 'Diencephalon - Anterior group of the posterior tubercular vmat2 neurons', 1002, 'Diencephalon - Anterior group of the posterior tubercular vmat2 neurons_slice002.png', 391)
(2, 'Diencephalon - Anterior group of the posterior tubercular vmat2 neurons', 1002, 'Diencephalon - Anterior group of the posterior tubercular vmat2 neurons_slice003.png', 391)
(3, 'Diencephalon - Anterior group of the posterior tubercular vmat2 neurons', 1002, 'Diencephalon - Anterior group of the posterior tubercular vmat2 neurons_slice004.png', 391)
(4, 'Diencephalon - Anterior group of the posterior tubercular vmat2 neurons', 1002, 'Diencephalon - Anterior group of the posterior tubercular vmat2 neurons_slice005.png', 391)
(5, 'Diencephalon - Anterior group of the posterior tubercular vmat2 neurons', 1002, 'Diencephalon - Anterior 

In [40]:
print len(rows)

40572
